In [1]:
import pickle
from pathlib import Path
import sys

import pandas as pd
from sklearn.svm import SVC
from xgboost import XGBClassifier

rank = 0 
#sys.path[0] = f'{Path().resolve().parents[rank]}' # mymoduleが上の階層にある場合rankを調整してコメント解除
from module.mymodule import PipeLine, df_copy, best_parameters
from module.kayano import age_categolize, stSlope_categolize, cholesterol_mean

### オリジナルデータの呼び出し

In [2]:
df = pd.read_csv('./data/train.csv')

### パイプライン候補の準備

In [3]:
def pipe_1(df, train_flg, split_kwrg):
    pipe = PipeLine()
    pipe.train_flg = train_flg
    pipe(df)
    pipe.standard_scaler()
    return pipe.df_num, pipe.df_target  # ここの出力を書き換える必要あり

def pipe_2(df, train_flg, split_kwrg):
    pipe = PipeLine()
    pipe.train_flg = train_flg
    pipe(df)
    pipe.df_num = df_copy(pipe.df_num, stSlope_categolize(pipe.df_cat), 'ST_Slope')
    pipe.standard_scaler()
    return pipe.df_num, pipe.df_target  # ここの出力を書き換える必要あり

def pipe_3(df, train_flg, split_kwrg):
    pipe = PipeLine()
    pipe.train_flg = train_flg
    pipe.train_flg = train_flg
    pipe(df)
    pipe.df_num = cholesterol_mean(pipe.df_num)
    pipe.standard_scaler()
    return pipe.df_num, pipe.df_target  # ここの出力を書き換える必要あり

### データセットの作成

In [4]:
split_kwrg = {}
train_flg = True

pipe_lines = [pipe_1, pipe_2, pipe_3]  # 特徴量候補を設定
data_set = {pipe.__name__: pipe(df, train_flg, split_kwrg) for pipe in pipe_lines}

### グリットサーチで訓練したモデルの呼び出し

In [5]:
with open('./data/model.pkl', 'rb') as f:
    trained_models = pickle.load(f)

### 最適なパラメータで再訓練

In [6]:
best_param = best_parameters(trained_models, pipe_lines)
retrain = [XGBClassifier, SVC]

retrained = {}
for re_tr in retrain:
    retrained[re_tr.__name__] = {}
    for pipe in pipe_lines:
        x, y = data_set[pipe.__name__]
        param = best_param[re_tr.__name__][pipe.__name__]
        model = re_tr(**param)
        model.fit(x.values, y.values.reshape(-1))
        retrained[re_tr.__name__][pipe.__name__] = model

with open('retrained_model.pkl', 'wb') as f:
    pickle.dump(retrained, f)